In [13]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [14]:
# Load Plant 1 data
gen = pd.read_csv(r"C:\Users\User\Desktop\solar_code\Task\plant2\Plant_2_Generation_Data.csv")



In [15]:
# Drop unnecessary columns
gen_clean = gen.drop(columns=['PLANT_ID', 'SOURCE_KEY', 'DAILY_YIELD', 'TOTAL_YIELD'])


In [16]:
# Ensure DATE_TIME is datetime
gen_clean['DATE_TIME'] = pd.to_datetime(gen_clean['DATE_TIME'])

In [17]:
gen_clean

,DATE_TIME,DC_POWER,AC_POWER
0,2020-05-15 00:00:00,0.0,0.0
1,2020-05-15 00:00:00,0.0,0.0
2,2020-05-15 00:00:00,0.0,0.0
3,2020-05-15 00:00:00,0.0,0.0
4,2020-05-15 00:00:00,0.0,0.0
...,...,...,...
67693,2020-06-17 23:45:00,0.0,0.0
67694,2020-06-17 23:45:00,0.0,0.0
67695,2020-06-17 23:45:00,0.0,0.0
67696,2020-06-17 23:45:00,0.0,0.0


In [18]:
# Group by timestamp and sum across inverters
gen_sum = (
    gen_clean.groupby('DATE_TIME')[['DC_POWER','AC_POWER']]
    .sum()
    .reset_index()
    .sort_values('DATE_TIME')   # ensure ascending order
)

# Reset index after sorting
gen_sum = gen_sum.reset_index(drop=True)

# Show first few rows
print(gen_sum.head(10))
print(gen_sum.tail(10))

            DATE_TIME  DC_POWER  AC_POWER
0 2020-05-15 00:00:00       0.0       0.0
1 2020-05-15 00:15:00       0.0       0.0
2 2020-05-15 00:30:00       0.0       0.0
3 2020-05-15 00:45:00       0.0       0.0
4 2020-05-15 01:00:00       0.0       0.0
5 2020-05-15 01:15:00       0.0       0.0
6 2020-05-15 01:30:00       0.0       0.0
7 2020-05-15 01:45:00       0.0       0.0
8 2020-05-15 02:00:00       0.0       0.0
9 2020-05-15 02:15:00       0.0       0.0
               DATE_TIME  DC_POWER  AC_POWER
3249 2020-06-17 21:30:00       0.0       0.0
3250 2020-06-17 21:45:00       0.0       0.0
3251 2020-06-17 22:00:00       0.0       0.0
3252 2020-06-17 22:15:00       0.0       0.0
3253 2020-06-17 22:30:00       0.0       0.0
3254 2020-06-17 22:45:00       0.0       0.0
3255 2020-06-17 23:00:00       0.0       0.0
3256 2020-06-17 23:15:00       0.0       0.0
3257 2020-06-17 23:30:00       0.0       0.0
3258 2020-06-17 23:45:00       0.0       0.0


In [19]:
gen_sum

,DATE_TIME,DC_POWER,AC_POWER
0,2020-05-15 00:00:00,0.0,0.0
1,2020-05-15 00:15:00,0.0,0.0
2,2020-05-15 00:30:00,0.0,0.0
3,2020-05-15 00:45:00,0.0,0.0
4,2020-05-15 01:00:00,0.0,0.0
...,...,...,...
3254,2020-06-17 22:45:00,0.0,0.0
3255,2020-06-17 23:00:00,0.0,0.0
3256,2020-06-17 23:15:00,0.0,0.0
3257,2020-06-17 23:30:00,0.0,0.0


In [20]:
type(gen_sum)

pandas.core.frame.DataFrame

In [21]:
# Ensure datetime is in datetime format
gen_sum['DATE_TIME'] = pd.to_datetime(gen_sum['DATE_TIME'])

# Create full expected timeline (15-min intervals)
full_time_index = pd.date_range(
    start = gen_sum['DATE_TIME'].min(),
    end   = gen_sum['DATE_TIME'].max(),
    freq='15T'
)

C:\Users\User\AppData\Local\Temp\ipykernel_29704\4283563040.py:5: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  full_time_index = pd.date_range(


In [22]:
len(full_time_index)

3264

In [23]:
# Find missing in plant_sum
missing_in_gen = full_time_index.difference(gen_sum['DATE_TIME'])


In [24]:
print("Expected timestamps:", len(full_time_index))
print("Plant Sum rows:", len(gen_sum))


print("\n❌ Missing in Generation:", len(missing_in_gen))
print(missing_in_gen[:10])   # show first 10

Expected timestamps: 3264
Plant Sum rows: 3259

❌ Missing in Generation: 5
DatetimeIndex(['2020-05-15 23:15:00', '2020-05-19 16:00:00',
               '2020-05-29 16:00:00', '2020-06-01 17:45:00',
               '2020-06-03 14:00:00'],
              dtype='datetime64[ns]', freq=None)


In [28]:
# Reindex to full 15-min timeline
gen_sum = (
    gen_sum
    .set_index('DATE_TIME')           # make datetime the index
    .reindex(full_time_index)         # add missing timestamps
    .reset_index()                    # bring DATE_TIME back as a column
    .rename(columns={'index':'DATE_TIME'})
)

print("Shape after reindexing:", gen_sum.shape)  # should be 3264


Shape after reindexing: (3264, 3)


In [29]:
# Interpolate missing values (linear between t-1 and t+1)
gen_sum[['DC_POWER','AC_POWER']] = gen_sum[['DC_POWER','AC_POWER']].interpolate(method='linear')


In [30]:
print("Final rows:", len(gen_sum))

Final rows: 3264


In [31]:
# Save cleaned generation data
gen_sum.to_csv(
    r"C:\Users\User\Desktop\solar_code\Task\plant2\Plant2_Generation_Cleaned.csv",
    index=False
)

print("✅ Generation data saved successfully!")


✅ Generation data saved successfully!
